# Продвинутый ООП. Продолжение

## 0. План занятия
* Познакомиться с основными магическими (dunder) методами Python.
* Узнать, как переопределять арифметические операции (`__add__`, `__sub__`, …).
* Узнать, как переопределять операции сравнения (`__eq__`, `__lt__`, …).
* Познакомиться с декоратором `functools.total_ordering` для упрощённого сравнения объектов.
* Освоить магические методы итераций (`__iter__`, `__next__`).
* Научиться писать контекстные менеджеры через `__enter__` / `__exit__`.

## 1. Введение в магические методы

### 1.1 Что такое *dunder*-методы

**Dunder-методы** (от *dou*ble **under**score) — специальные методы классов Python, чьи имена **начинаются и заканчиваются двумя подчёркиваниями**.  
Они определяют, как ваш объект должен вести себя в стандартных конструкциях языка: арифметика, сравнения, `len()`, цикл `for`, контекстный менеджер и т.д.



| Синтаксис Python | Какой метод вызывается | Что настраивается |
|------------------|-----------------------|-------------------|
| `obj + other`    | `obj.__add__(other)`  | Арифметические операции |
| `len(seq)`       | `seq.__len__()`       | Размер/длина объекта |
| `for x in seq:`  | `iter(seq)` → `seq.__iter__()` | Итерации |
| `a == b`         | `a.__eq__(b)`         | Проверка равенства |
| `with res:`      | `res.__enter__()` / `res.__exit__()` | Контекстный менеджер |


> 💡 **Мы почти никогда не вызываем эти методы напрямую**.  
> Вместо этого мы *переопределяем* их, чтобы встроенный синтаксис Python «сам» выполнял нужное действие для наших объектов.



#### Мини-пример

In [5]:
class Vector2D:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __add__(self, other):
        return Vector2D(self.x + other.x, self.y + other.y)

    def __repr__(self):
        return f"Vector2D({self.x}, {self.y})"

In [ ]:
self.__add__(other)

In [6]:
v1 = Vector2D(1, 2)
v2 = Vector2D(3, 4)
print(v1 + v2)        # Vector2D(4, 6)

Vector2D(4, 6)


Перегрузка `__add__` делает код **короче и читабельнее**, что особенно полезно в ML-проекте при работе, к примеру, с векторами градиентов, метриками или конфигурациями моделей.

### 1.2 Зачем знать о *dunder*-методах разработчику ML-проектов

| Причина | Короткое объяснение | Один показательный пример |
|---------|--------------------|---------------------------|
| **Читабельный DSL** | Позволяют писать «формулы» и операции над моделями/тензорами так же естественно, как с числами. | `total_loss = ce_loss + l2_loss * 0.001` — работает, потому что в классах лоссов переопределён `__add__` и `__mul__`. |
| **Интеграция с популярными библиотеками** | Многие API (`torch.utils.data.DataLoader`, `sklearn` пайплайны) требуют конкретных *dunder*-методов. | Свой класс `CustomDataset` станет совместим с `DataLoader`, если реализовать `__len__` и `__getitem__`. |
| **Меньше «клеевого» кода** | Один переопределённый метод даёт поддержку целой группы операций без дополнительных функций. | Реализовав всего `__iter__` и `__next__`, вы автоматически получаете работу с циклом `for`, list-comprehensions и генераторами. |
| **Безопасное управление ресурсами** | Контекстные менеджеры через `__enter__` / `__exit__` гарантируют освобождение памяти или файлов. | `with gpu_session(): train_model()` освобождает GPU-ресурсы даже если обучение упало по исключению. |

## 2. Арифметические операции

### 2.1 Обзор арифметических *dunder*-методов  


| Оператор | Левосторонний метод | Правосторонний метод<sup>1</sup> | Что вернут методы |
|----------|--------------------|----------------------------------|-------------------|
| `a + b`  | `a.__add__(b)`     | `b.__radd__(a)`                  | Новый объект (желательно неизменяемый) **или** `NotImplemented` |
| `a - b`  | `a.__sub__(b)`     | `b.__rsub__(a)`                  | то же |
| `a * b`  | `a.__mul__(b)`     | `b.__rmul__(a)`                  | то же |
| `a / b`  | `a.__truediv__(b)` | `b.__rtruediv__(a)`              | то же |

<sup>1</sup> Python вызывает правостороннюю версию только если левый операнд вернул `NotImplemented` **или** не реализует операцию вовсе.  

**Короткие правила реализации**

1. **Не стоит изменять** левый и правый операнд - лучше возвращать новый объект.  
2. Если тип `other` неподдерживаемый → `return NotImplemented`.  
3. Для симметричных операций (`+`, `*`) просто делегируйте работу в `__radd__`, `__rmul__`:

```python
def __radd__(self, other):
    return self.__add__(other)
```





#### Пример: минимальный `Vector`

In [11]:
class Vector:
    def __init__(self, *coords):
        self.coords = coords

    # v + w
    def __add__(self, other):
        if not isinstance(other, Vector):
            return NotImplemented
        
        summed = (a + b for a, b in zip(self.coords, other.coords))
        return Vector(*summed)

    # w + v (если w не знает, как сложить Vector)
    __radd__ = __add__

    def __repr__(self):
        return f"Vector{self.coords}"

In [ ]:
v1 = Vector(1, 2, 3)
v2 = Vector(4, 5, 6)

print(v1 + v2)   # Vector(5, 7, 9)
print(v2 + v1)   # Vector(5, 7, 9) — работает благодаря __radd__

Vector(5, 7, 9)
Vector(5, 7, 9)


Такой класс уже поддерживает читаемую арифметику, а добавив `__sub__`, `__mul__`, `__truediv__` (в том же стиле) можно писать выражения наподобие `momentum * grad1 - grad2`.

### 2.2 Пример: класс `Vector` для градиентов

> **Зачем?** Градиент модели — это просто вектор весовых производных.  
> Если такой вектор «умеет» в арифметику, код обучения становится читабельным:

```python
weights -= lr * grad          # вместо for-loop’ов по координатам
```


#### Реализация класса `Vector`

In [ ]:
from math import sqrt

class Vector:
    """Вектор произвольной длины"""
    
    def __init__(self, *coords: float):
        if not coords:
            raise ValueError("Vector must contain at least one coordinate")
        self._coords = tuple(float(c) for c in coords)

    # ——— Представление ———
    def __repr__(self):
        return f"Vector{self._coords}"
    
    # ——— Итерации и len() ———
    def __iter__(self):
        return iter(self._coords)
    
    def __len__(self):
        return len(self._coords)
    
    # ——— Арифметика ———
    def _check_other(self, other: "Vector"):
        if not isinstance(other, Vector) or len(other) != len(self):
            return NotImplemented
        return other

    def __add__(self, other):
        other = self._check_other(other)
        if other is NotImplemented:
            return NotImplemented
        return Vector(*(a + b for a, b in zip(self, other)))

    __radd__ = __add__

    def __sub__(self, other):
        other = self._check_other(other)
        if other is NotImplemented:
            return NotImplemented
        return Vector(*(a - b for a, b in zip(self, other)))

    def __mul__(self, k: float):
        if not isinstance(k, (int, float)):
            return NotImplemented
        return Vector(*(a * k for a in self))
    
    __rmul__ = __mul__

    def __truediv__(self, k: float):
        if not isinstance(k, (int, float)):
            return NotImplemented
        return Vector(*(a / k for a in self))

    # Полезные свойства
    @property
    def norm(self) -> float:
        return sqrt(sum(a * a for a in self))

#### Использование в обновлении весов

In [22]:
# «Сырые» данные
grad1 = Vector(0.12, -0.03, 0.20)
grad2 = Vector(0.08, -0.04, 0.18)
weights = Vector(2.0, -1.5, 0.3)
lr = 0.01

In [23]:
# Считаем средний градиент по двум батчам
avg_grad = (grad1 + grad2) * 0.5          # Vector(0.10, -0.035, 0.19)
avg_grad

Vector(0.1, -0.035, 0.19)

In [24]:
# Шаг градиентного спуска
new_weights = weights - lr * avg_grad     # Vector(1.999, -1.49965, 0.2981)

# print("‖avg_grad‖ =", round(avg_grad.norm, 4))
print("new weights:", new_weights)

new weights: Vector(1.999, -1.49965, 0.2981)


*Получаем лаконичный, «математичный» код без ручных циклов — именно за счёт перегрузки `__add__`, `__sub__`, `__mul__`, `__truediv__`, `__rmul__`.*

## 3. Сравнение объектов и `total_ordering`

### 3.1 Методы для сравнения: `__eq__`, `__lt__`, `__gt__`, `__le__`, `__ge__`

| Оператор | Метод, который вызывает Python | Что должен вернуть метод |
|----------|--------------------------------|--------------------------|
| `a == b` | `a.__eq__(b)`                  | `True` / `False` **или** `NotImplemented` |
| `a <  b` | `a.__lt__(b)`                  | то же |
| `a >  b` | `a.__gt__(b)`                  | то же |
| `a <= b` | `a.__le__(b)`                  | то же |
| `a >= b` | `a.__ge__(b)`                  | то же |


**Короткие правила**

1. Всегда стоит проверить тип аргумента: если он неподдерживаем, возвращаем `NotImplemented` — это позволит Python попробовать правосторонний метод (`b.__gt__(a)` и т.д.).
2. Логика всех пяти операций должна быть **согласованной**; несогласованность ломает сортировки и множества.
3. Если объект упорядочивается по одному критерию, достаточно реализовать `__eq__` и **одно** из «строгих» сравнений, а остальные сгенерировать декоратором `functools.total_ordering` (разберём в § 3.2).

#### Пример: метрика качества

In [32]:
class Metric:
    """
    Обёртка над значением метрики.
    «Больше» - значит лучше (accuracy, ROC-AUC и т.п.).
    """
    def __init__(self, name: str, value: float):
        self.name = name
        self.value = float(value)

    def __eq__(self, other):
        if not isinstance(other, Metric):
            return NotImplemented
        
        return self.value == other.value

    def __lt__(self, other):
        if not isinstance(other, Metric):
            return NotImplemented
        
        return self.value < other.value    # «меньше» = хуже

    def __repr__(self):
        return f"{self.name}: {self.value:.3f}"

In [33]:
acc_1 = Metric("Accuracy", 0.88)
acc_2 = Metric("Accuracy", 0.89)

roc_auc = Metric("ROC-AUC", 0.91)
roc_auc_2 = Metric("ROC-AUC", 0.92)

print(acc_1 == acc_2)       # False
print(acc_1 < acc_2)        # True
print(roc_auc < roc_auc_2)  # True

False
True
True


In [34]:
print(acc_1)       # Accuracy: 0.880

Accuracy: 0.880


Одних `__eq__` + `__lt__` достаточно — остальные операции (`>`, `>=`, `<=`) сгенерированы `total_ordering`, и объект корректно работает с `max`, `sorted`, `set`.

### 3.2 Декоратор `functools.total_ordering`

`functools.total_ordering` — это «ленивый» способ получить полный набор операций сравнения, написав **только два метода**:

* `__eq__(self, other)`  
* **и** один из строго-порядковых: `__lt__`, `__le__`, `__gt__` или `__ge__`

Декоратор автоматически сгенерирует остальные три.



In [28]:
from functools import total_ordering

@total_ordering
class Experiment:
    """
    Храним id эксперимента и метрику (чем выше метрика тем лучше).
    Реализуем __eq__ и __lt__ - остальное достроит декоратор total_ordering.
    """
    def __init__(self, exp_id: str, score: float):
        self.exp_id = exp_id
        self.score = score

    def __eq__(self, other):
        if not isinstance(other, Experiment):
            return NotImplemented
        return self.score == other.score

    def __lt__(self, other):
        if not isinstance(other, Experiment):
            return NotImplemented
        return self.score < other.score      # «меньше» = хуже

    def __repr__(self):
        return f"{self.exp_id}: {self.score:.3f}"

In [29]:
exp_1 = Experiment("run-42", 0.875)
exp_2 = Experiment("run-43", 0.891)

print(exp_1 < exp_2)      # True   (__lt__)
print(exp_1 >= exp_2)     # False  (__ge__ сгенерирован автоматически)


True
False


In [ ]:
print(max(exp_1, exp_2))

run-43: 0.891


**Плюсы**

* Экономия кода: не пишем дублирующие методы.  
* Труднее сделать логическую ошибку — достаточно проверить согласованность `__eq__` и одного «строгого» сравнения.

**Минусы / ограничения**

1. Декоратор **не проверяет** ваши методы на корректность — ответственность за логику остаётся на вас.  
2. Все вычисления, необходимые для сравнения, будут выполняться при каждом вызове сгенерённых методов; если они дорогие, кешируйте результат вручную.

### 3.3 Пример: класс `Metric` с возможностью сортировки по значению


In [36]:
from functools import total_ordering

@total_ordering
class Metric:
    """
    Обёртка над значением метрики ― «чем больше, тем лучше».
    Достаточно переопределить __eq__ и __lt__; остальные операции
    сгенерирует total_ordering.
    """
    def __init__(self, name: str, value: float):
        self.name, self.value = name, float(value)

    # равенство - одинаковые значения
    def __eq__(self, other):
        if not isinstance(other, Metric):
            return NotImplemented
        return self.value == other.value

    # «меньше» - хуже (используем для сортировки/max)
    def __lt__(self, other):
        if not isinstance(other, Metric):
            return NotImplemented
        return self.value < other.value

    def __repr__(self):
        return f"{self.name}: {self.value:.3f}"


In [38]:
scores = [
    Metric("Accuracy", 0.88),
    Metric("ROC-AUC", 0.91),
    Metric("F1",       0.84),
]

best = max(scores)          # → ROC-AUC: 0.910
print("Best metric -", best)

# От худшей к лучшей (ascending)
print("Sorted (asc):", sorted(scores))

# От лучшей к худшей (descending)
print("Sorted (desc):", sorted(scores, reverse=True))

Best metric - ROC-AUC: 0.910
Sorted (asc): [F1: 0.840, Accuracy: 0.880, ROC-AUC: 0.910]
Sorted (desc): [ROC-AUC: 0.910, Accuracy: 0.880, F1: 0.840]



*За счёт `total_ordering` объект сразу работает с `max`, `min`, `sorted`, `heapq` и любыми структурами, где важен естественный порядок элементов.*

## 4. Итерационный протокол

### 4.1 `__iter__` и `__next__`

| Что нужно знать | Коротко |
|-----------------|---------|
| **`__iter__(self)`** | Должен вернуть **итератор** — объект, у которого есть метод `__next__`. Чаще всего — *самого себя*. |
| **`__next__(self)`** | Возвращает следующий элемент последовательности. Когда элементов больше нет, поднимает `StopIteration`. |
| **Итератор ≠ Iterable** | *Iterable* — то, что можно передать в `iter()`. *Iterator* — конкретный объект, по которому «ходит» цикл. |
| **Цикл `for`** | Внутри превращается в: `it = iter(obj)` → `while True: val = next(it)` … |



### 4.2 Пример: `MiniBatchIterator` для генерации батчей

In [ ]:
class MiniBatchIterator:
    """Итерируем по списку данных кусками фиксированного размера."""
    def __init__(self, data, batch_size):
        self.data, self.batch_size = data, batch_size

    def __iter__(self):
        # print("iter self._idx =", self._idx)
        self._idx = 0          # сбрасываем каждый раз при новом iter()
        return self

    def __next__(self):
        # print("next self._idx =", self._idx)
        # print(len(self.data))
        if self._idx >= len(self.data):
            raise StopIteration
        batch = self.data[self._idx : self._idx + self.batch_size]
        self._idx += self.batch_size
        return batch


In [52]:
batches = MiniBatchIterator(range(10), batch_size=3)

for batch in batches:
    print(list(batch))       # [0,1,2]  [3,4,5]  [6,7,8]  [9]

[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
[9]


In [50]:
lst = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
lst[9:12]

[9]

Такой объект сразу совместим с любыми функциями, ожидающими iterable: `list(batches)`, `sum(len(b) for b in batches)`, а в ML-коде - с обучающим циклом `for X, y in loader:`.

In [53]:
sum(len(b) for b in batches)

10

## 5. Контекстные менеджеры

### 5.0 Контекстный менеджер: что это и зачем они в ML-коде

*Контекстный менеджер* — объект, который реализует пары методов `__enter__ / __exit__`.  
Используется через конструкцию `with …:`, чтобы **автоматически и надёжно** управлять ресурсами.

| Что делает `with` | Почему это важно |
|-------------------|------------------|
| Гарантирует, что ресурс будет закрыт, даже если внутри блока возникнет исключение. | Устраняет «утечки» файлов, сокетов, GPU-памяти. |
| Сокращает «шум» кода: открытие/закрытие явно сгруппированы и находятся рядом. | Повышает читаемость экспериментов и пайплайнов. |
| Позволяет вложенные контексты — удобно комбинировать несколько ресурсов. | Например, файл + `torch.no_grad()` + таймер. |


#### Типичные случаи в машинном обучении и анализе данных

1. **Работа с БД**

   ```python
   with sqlite3.connect("db.sqlite") as conn:
       cursor = conn.cursor()
       cursor.execute("SELECT * FROM table")
       # соединение автоматически закрывается
   ```

2. **Отключение градиентов при инференсе**

   ```python
   with torch.no_grad():
       logits = model(X)        # никакой записи в граф
   ```

3. **Запись логов / метрик**

   ```python
   with summary_writer.as_default():       # TensorBoard
       tf.summary.scalar("loss", loss, step=step)
   # файл события надёжно закрыт
   ```

Контекстные менеджеры делают ML-код **короче, безопаснее и более декларативным**, избавляя от ручного `try/finally` и случайно забытых `close()`.

### 5.1 Протокол `with` — `__enter__` / `__exit__`

| Шаг внутри `with` | Что вызывает Python | Ваше назначение |
|-------------------|--------------------|-----------------|
| 1. Создаёт менеджер | `mgr = Context()` | ­ |
| 2. Входит в блок | `val = mgr.__enter__()` | Подготовить ресурс. Вернуть объект, который будет доступен как `as val` (часто — сам `mgr`). |
| 3. Выполняет тело | код внутри `with …:` | ­ |
| 4. Выходит из блока | `mgr.__exit__(exc_type, exc, tb)` | Освободить ресурс. Если вернуть **`True`**, подавить возникшее исключение; иначе оно всплывёт. |



#### Пример: измеряем время обучения c помощью `Timer`

In [56]:
import time

class Timer:
    def __enter__(self):
        self._start = time.perf_counter()
        return self            # передаём в блок `as t`
    
    def __exit__(self, exc_type, exc, tb):
        elapsed = time.perf_counter() - self._start
        print(f"Elapsed: {elapsed:.3f}s")
        # ничего не возвращаем

def train_model():
    time.sleep(0.3)          # имитация долгой работы
    print("Model trained")
    return True

In [ ]:
# использование
with Timer() as t:
    train_model()

Model trained
Elapsed: 0.300s


*Благодаря `__enter__` / `__exit__` ресурс (здесь — таймер, но это может быть файл, подключение к БД, GPU-сессия) гарантированно и безопасно «закрывается» даже при исключениях.*

### 5.3 Альтернатива: `contextlib.contextmanager`

Иногда писать полноценный класс с `__enter__ / __exit__` избыточно.  
`contextlib.contextmanager` позволяет создать **функцию-генератор**, которая превращается в контекстный менеджер «на лету».


In [58]:
from contextlib import contextmanager
import sys
import io

@contextmanager
def suppress_stdout():
    """Временно перенаправляем stdout в «чёрную дыру»."""
    old_stdout = sys.stdout
    sys.stdout = io.StringIO()      # 1 подготовка ресурса
    try:
        yield                       # <- точка, где выполняется тело with-блока
    finally:
        sys.stdout = old_stdout     # 2 очистка ресурса

In [59]:
def noisy_training_loop():
    print("Starting training...")
    time.sleep(1)
    print("Training in progress...")
    time.sleep(1)
    print("Training finished!")


with suppress_stdout():
    noisy_training_loop()  # все print’ы будут скрыты

**Ключевые моменты**

1. **До `yield`** — код, эквивалентный `__enter__`: готовим/выделяем ресурс.  
2. **После `yield`** — код, эквивалентный `__exit__`: освобождаем ресурс, даже если внутри было исключение.  
3. Всё, что передано в `yield <value>`, попадёт в `as <value>` блока `with`.

> ✔ Подходит для «одноразовых» простых менеджеров (таймер, подавление логов, временная смена директории).  
> ✖ Для сложных объектов с состоянием, наследованием или повторным использованием удобнее полноценный класс.

## 6. Итоги занятия

* Мы научились писать и переопределять магические методы для арифметики, сравнения, итераций и контекстного менеджера.
* Посмотрели примеры из ML‑контекста (векторы градиентов, метрики, батчи, таймеры).

## 7. Домашнее задание

| № | Тема | Постановка задачи | Ключевые проверки |
|---|------|------------------|-------------------|
| **1** | **`Point`** | Создайте класс точки `Point(x, y)` с поддержкой `__add__`, `__sub__`, умножения на **скаляр** (`p * k` и `k * p`). | ✔ `Point(1,2) + Point(3,4)` → `(4, 6)`.<br>✔ Работают правосторонние операции `__radd__`, `__rmul__`. |
| **2** | **`ModelScore` + `total_ordering`** | Класс `ModelScore` хранит `name` и `score`; «лучше» — **больше**. Реализуйте `__eq__` и `__lt__`, а остальные сравнения пусть сгенерирует `total_ordering`. | ✔ `sorted([...])` упорядочивает модели от худшего к лучшему.<br>✔ `max(models)` возвращает модель с наивысшим `score`. |
| **3** | **`EvenSequence`** | Итерабельный класс, представляющий **последовательность** чётных чисел от 0 до `n` (не включая). Должен поддерживать:<br>• цикл `for` (`__iter__`, `__next__`)<br>• функцию `len()`<br>• индексирование `obj[i]`. | ✔ `for x in EvenSequence(9)` выводит `0 2 4 6 8`.<br>✔ `len(EvenSequence(10))` возвращает `5`.<br>✔ `EvenSequence(12)[2]` → `4`; при выходе за пределы — `IndexError`. |
| **4** | **`FileStats` (context manager)** | Контекстный менеджер, который открывает текстовый файл и делает две вещи:<br>• предоставляет файл как `as f` внутри блока;<br>• при выходе печатает «Lines: _…_, Words: _…_», где _…_ — количество строк и слов в файле. | ✔ Файл доступен внутри `with` как объект-контекст (`with FileStats(path) as f:`).<br>✔ После выхода печатается корректная статистика.<br>✔ Файл закрывается даже при исключении в блоке. |

In [ ]:
file_txt = """Несколько строк текста для примера использования
контекстного менеджера.
Всё, это сообщение будет записано в файл example.txt."""

with open("example.txt", "w") as f:
    f.write(file_txt)      # записываем текст в файл

### 5★ SeededBatchLoader

| Шаг | Требование | Подсказки для проверки |
|-----|------------|------------------------|
| **1** | **Сигнатура**<br>`SeededBatchLoader(data: list, batch_size: int, shuffle: bool = True)` | В конструкторе сохраните `data`, `batch_size`, `shuffle`. Никаких сторонних импортов, кроме `import random`. |
| **2** | **Итератор-протокол**<br>• `__iter__` должен возвращать **самого себя** и сбрасывать внутренний индекс на 0.<br>• `__next__` отдаёт следующий батч; когда данные кончились — начинает новую «эпоху»: опционально перемешивает данные (если `shuffle=True`) и продолжает выдавать батчи. | Проверка: <br>`list(itertools.islice(loader, k))` выдаёт именно `k` батчей.<br>При `shuffle=False` порядок элементов повторяем. |
| **3** | **`__len__`**<br>Возвращает количество батчей в одной эпохе. | `len(SeededBatchLoader(range(10), 4))` → `3` (`[0-3]`, `[4-7]`, `[8-9]`). |
| **4** | **Контекст-менеджер**<br>`__enter__(seed)` / `__exit__`<br>• Сохраняет текущее состояние ГПСЧ: `state = random.getstate()`.<br>• Делает `random.seed(seed)`.<br>• Возвращает **самого себя**.<br>• В `__exit__` восстанавливает `random.setstate(state)` и не подавляет исключения. | Проверка: <br>`with SeededBatchLoader(data, 2, True) as l1` и тот же блок с тем же `seed` → одинаковые батчи.<br>После выхода из блока глобальный `random.randint(…)` даёт те же результаты, что и до `with`. |
| **5** | **Бесконечная генерация**<br>Итератор никогда не заканчивается: потребитель сам решает, сколько батчей взять (например, через `itertools.islice`). | `for i, batch in enumerate(loader): ... if i == 99: break` — цикл работает без ошибок. |

#### Подсказки по реализации

* В `__next__` храните индекс текущего элемента; когда он «перепрыгивает» через `len(data)`, сформируйте новую эпоху.
* Чтобы перемешать данные без NumPy, используйте:  
  ```python
  if self.shuffle:
      random.shuffle(self._data_copy)
  ```
  где `_data_copy` — копия исходного списка, чтобы не мутировать `self.data`.
* Старайтесь **не дублировать код**: например, вынесите логику старта новой эпохи в отдельный метод `_start_epoch()`.
